### Extract and analyis Photo data

In [280]:
import os
import hashlib
import glob
from PIL import Image
import PIL.ExifTags as ExifTags

import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

import pandas as pd
import sqlite3
#from sqlalchemy import create_engine #requires psycog2 module to connect to postgres SQL

https://github.com/frankcleary/img-exif/blob/master/exif_data.py


In [5]:
ls

1.jpg*                     Lightroom 5 Catalog.lrcat
2.arw*                     Untitled.ipynb
3.cr2*


In [6]:
# Extract exif data from photo 
def extractExif(photo_file):
    """Extract a dict of exif data from the photo file"""
    img = Image.open(photo_file)
    raw_exif = img._getexif()
    if raw_exif is None:
        return {}
    else:
        return {ExifTags.TAGS[k]: v for k, v in raw_exif.items()
                if k in ExifTags.TAGS}


In [7]:
extractExif('1.jpg')

{'ApertureValue': (262144, 65536),
 'Artist': 'Matthew Grant   Buffalo,NY      (716)6226771',
 'BodySerialNumber': '172028005195',
 'CameraOwnerName': '',
 'ColorSpace': 1,
 'ComponentsConfiguration': b'\x01\x02\x03\x00',
 'Copyright': 'NO repduction without written consent',
 'CustomRendered': 0,
 'DateTime': '2015:12:22 02:50:18',
 'DateTimeDigitized': '2015:12:22 02:50:18',
 'DateTimeOriginal': '2015:12:22 02:50:18',
 'ExifImageHeight': 2560,
 'ExifImageWidth': 3840,
 'ExifInteroperabilityOffset': 9326,
 'ExifOffset': 360,
 'ExifVersion': b'0230',
 'ExposureBiasValue': (0, 1),
 'ExposureMode': 1,
 'ExposureProgram': 1,
 'ExposureTime': (1, 50),
 'FNumber': (4, 1),
 'Flash': 16,
 'FlashPixVersion': b'0100',
 'FocalLength': (34, 1),
 'FocalPlaneResolutionUnit': 2,
 'FocalPlaneXResolution': (3840000, 1461),
 'FocalPlaneYResolution': (2560000, 972),
 'GPSInfo': {0: b'\x02\x03\x00\x00'},
 'ISOSpeedRatings': 6400,
 'LensModel': 'EF17-40mm f/4L USM',
 'LensSerialNumber': '0000000000',
 'Le

In [26]:
#find photos file path in base folder
photos = []
def getPhotos(base_dir):
    types = ('*.jpg', '*.arw','*.cr2',)
    for filename in types:
        photos = glob.glob(base_dir)
        print(photos)
        
        

In [27]:
getPhotos('../')

['../']
['../']
['../']


###  Lightroom catalog database 
#### query, analyse and visualize photo data 

* word cloud (images per keyword)
* images per focal length
* images per camera
* images per lens
* rated images 
* resolutions 
* file formats
* image timeline
* images by day,month,year


In [16]:
# connect to sqlite db 
db = sqlite3.connect("Lightroom 5 Catalog.lrcat")
c = db.cursor()

In [296]:
c.execute("SELECT name FROM sqlite_master WHERE type='table' and name like '%image%'")

In [294]:
# Get database table names
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' and name like '%image%'", db)
tables.head()
# 

name
0                 Adobe_imageProperties
1     Adobe_libraryImageDevelopSnapshot
2  Adobe_libraryImageDevelopHistoryStep
3              Adobe_imageProofSettings
4                 AgLibraryKeywordImage

In [68]:
# list of paths, file name and extensions
pd.read_sql_query("select fi.id_local, pathFromRoot, baseName, extension " +\
                  "from AgLibraryFile fi " +\
                  "join AgLibraryFolder fo on fi.folder = fo.id_local",db ).head()


id_local pathFromRoot  baseName extension
0      2751       07-30/  _MG_0007       CR2
1      2762       07-30/  _MG_0009       CR2
2      2773       07-30/  _MG_0010       CR2
3      2784       07-30/  _MG_0014       CR2
4      2795       07-30/  _MG_0020       CR2

In [ ]:
pd.read_sql_query("select * from AgLibraryFile ")

In [231]:
# image count by file format
pd.read_sql_query("select count(*) as Count,lc_idx_filenameExtension fileFormat " +\
                  "from AgLibraryFile fi " +\
                  "join AgLibraryFolder fo on fi.folder = fo.id_local " +\
                  "group by lc_idx_filenameExtension " +\
                  "order by Count DESC",db )

Count fileFormat
0  10278        cr2
1   7358        jpg
2    703        mov
3    151        arw
4     54        tif
5     19        png
6     16        psd
7      5        mp4

In [147]:
# Number of images with keyword "Portrait" 
pd.read_sql_query("Select count(image) from aglibrarykeywordimage where tag=(" +\
                  "SELECT id_local from aglibrarykeyword where lc_name='portrait')",db ).head()

count(image)
0            36

In [148]:
# list of keywords 
pd.read_sql_query("SELECT id_local,name from aglibrarykeyword order by name",db ).head()

id_local         name
0        20         None
1    336886          #52
2    521507         2014
3    561960  4th of July
4    521508          ADK

In [194]:
# top ten keywords used by image count 
pd.read_sql_query("SELECT count(tag),name from aglibrarykeywordimage keywordimg " +\
                  "join aglibrarykeyword keyword on keywordimg.tag = keyword.id_local " +\
                  "group by name " +\
                  "order by 1 DESC " +\
                  "limit 10",db )

count(tag)             name
0        4990           Hockey
1        4952   Buffalo Sabres
2        2783         climbing
3        2447           Bishop
4        1817    rock climbing
5        1334      4th of July
6        1334           Summer
7        1103     Niagara Glen
8         846  Red River Gorge
9         784          America

In [58]:
# most recent photo in catalog
pd.read_sql_query("SELECT max(captureTime) FROM adobe_images",db ).head()

max(captureTime)
0  2015-05-22T12:56:28

In [269]:
# image count by months? 
pd.read_sql_query("SELECT count(*) as Count ,strftime('%Y-%m', captureTime) YearMonth FROM adobe_images " +\
                  "group by strftime('%Y-%m', captureTime) "
                  "order by captureTime DESC",db ).head()

Count YearMonth
0     74   2015-05
1    162   2015-04
2     22   2015-03
3     12   2014-12
4    472   2014-11

In [233]:
pd.read_sql_query("Select count(*) from adobe_images " +\
                   "group by " +\
                   "order by captureTime ",db).head()

DatabaseError: Execution failed on sql 'Select count(*) from adobe_images group by order by captureTime ': near "order": syntax error

In [70]:
pd.read_sql_query("select rf.absolutePath || fo.pathFromRoot || fi.baseName || '.' || fi.extension as fullName, " +\
                   "i.fileHeight, i.fileWidth, i.captureTime, md.xmp " +\
                   "from Adobe_images i " +\
                   "join AgLibraryFile fi on i.rootFile = fi.id_local " +\
                   "join AgLibraryFolder fo on fi.folder = fo.id_local " +\
                   "join AgLibraryRootFolder rf on fo.rootFolder = rf.id_local " +\
                   "join Adobe_AdditionalMetadata md on md.image = i.id_local " +\
                    "order by i.captureTime desc",db ).head()

fullName  fileHeight  fileWidth  \
0  /Volumes/LaCie/2015/2015 May 22/RRG-20150522-7...      3648.0     5472.0   
1  /Volumes/LaCie/2015/2015 May 22/RRG-20150522-7...      3648.0     5472.0   
2  /Volumes/LaCie/2015/2015 May 22/RRG-20150522-7...      3648.0     5472.0   
3  /Volumes/LaCie/2015/2015 May 22/RRG-20150522-7...      3648.0     5472.0   
4  /Volumes/LaCie/2015/2015 May 22/RRG-20150522-7...      3648.0     5472.0   

           captureTime                                                xmp  
0  2015-05-22T12:56:28  <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="A...  
1  2015-05-22T12:54:38  <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="A...  
2  2015-05-22T12:54:34  <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="A...  
3  2015-05-22T12:25:31  <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="A...  
4  2015-05-22T12:24:47  <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="A...

In [218]:
# image count by rating 
pd.read_sql_query("select count(*) as Count,rating from Adobe_images " +\
                   "group by rating " +\
                   "order by Count ASC ",db)

Count  rating
0      3     5.0
1     29     4.0
2    407     3.0
3   1254     2.0
4   4121     1.0
5  13008     NaN

In [237]:
# image metadata
pd.read_sql_query("select * from AgHarvestedExifMetadata order by id_local",db).head()

id_local  image  aperture  cameraModelRef  cameraSNRef  dateDay  dateMonth  \
0      2759   2749       4.0            32.0         33.0     30.0        7.0   
1      2770   2761       4.0            32.0         33.0     30.0        7.0   
2      2781   2772       4.0            32.0         33.0     30.0        7.0   
3      2792   2783       4.0            32.0         33.0     30.0        7.0   
4      2803   2794       4.0            32.0         33.0     30.0        7.0   

   dateYear  flashFired  focalLength  gpsLatitude  gpsLongitude  gpsSequence  \
0    2011.0         0.0         17.0          NaN           NaN          0.0   
1    2011.0         0.0         17.0          NaN           NaN          0.0   
2    2011.0         0.0         17.0          NaN           NaN          0.0   
3    2011.0         0.0         40.0          NaN           NaN          0.0   
4    2011.0         0.0         40.0          NaN           NaN          0.0   

   hasGPS  isoSpeedRating  lensRef  shutterSpeed  
0       0           100.0   2294.0      5.906891  
1       0           200.0   2294.0      4.906891  
2       0           160.0   2294.0      4.906891  
3       0           100.0   2294.0      8.965784  
4       0           100.0   2294.0      8.643856

In [290]:
# image count by focal length 
pd.read_sql_query("select count(id_local)as Count,focalLength from AgHarvestedExifMetadata " +\
                  "group by focalLength " +\
                  "order by focalLength DESC",db).head()

Count  focalLength
0   3405        300.0
1    214        280.0
2      9        273.0
3      3        265.0
4     17        260.0

In [291]:
# image count by aperture size 
pd.read_sql_query("select count(id_local)as Count,aperture from AgHarvestedExifMetadata " +\
                  "group by aperture " +\
                  "order by aperture DESC ",db)


Count  aperture
0       1  9.715962
1      18  8.918863
2       9  8.643856
3      16  8.339850
4      32  8.000000
5      18  7.614710
6      36  7.400879
7     133  6.918863
8      70  6.643856
9     272  6.339850
10    267  6.000000
11    478  5.655638
12    336  5.310704
13   1209  4.970854
14    548  4.643856
15    965  4.339850
16   2603  4.000000
17    838  3.614710
18   2206  3.356144
19   6765  2.970854
20      1  2.865919
21    119  2.643856
22     76  2.275007
23    119  2.000000
24    236  1.695994
25    252  1.356144
26    531  0.970854
27    668       NaN

In [191]:
# image count by camera used
pd.read_sql_query("select count(*) as Count,cameramodel.value from AgHarvestedExifMetadata metadata " +\
                  "join AgInternedExifCameraModel cameramodel on metadata.cameraModelRef = cameramodel.id_local " +\
                  "group by cameramodel.id_local " +\
                  "order by 1 DESC",db)

Count                  value
0    9007           Canon EOS 7D
1    6400  Canon EOS 5D Mark III
2    1773   Canon EOS-1D Mark IV
3     681    Hero3-Black Edition
4     259            DSC-RX100M3
5      27  Canon EOS-1D Mark III
6      13           ScanJet 3970
7      11           MP240 series
8       7   Canon PowerShot A630
9       2                  Droid
10      1              X-3,C-60Z
11      1    Canon EOS REBEL T3i
12      1               SGH-I717

In [192]:
# image count by lens used
pd.read_sql_query("select count(*) as Count,exiflens.value from AgHarvestedExifMetadata metadata " +\
                  "join AgInternedExifLens exiflens on metadata.lensref = exiflens.id_local " +\
                  "group by exiflens.id_local " +\
                  "order by 1 DESC",db)

Count                        value
0    6061        EF70-200mm f/2.8L USM
1    3405           EF17-40mm f/4L USM
2    3405        EF300mm f/2.8L IS USM
3    1702             EF50mm f/1.4 USM
4     764             EF40mm f/2.8 STM
5     705                      11-16mm
6     408  EF70-200mm f/2.8L USM +1.4x
7     314                         30mm
8     269  EF28-135mm f/3.5-5.6 IS USM
9     151             24-70mm F1.8-2.8
10    142      EF16-35mm f/2.8L II USM
11    108        8.8-25.7 mm f/1.8-2.8
12     14      EF100mm f/2.8 Macro USM
13      7                  7.3-29.2 mm
14      6  EF-S18-55mm f/3.5-5.6 IS II

In [293]:
# focalLength, camera, lens 
pd.read_sql_query("SELECT focalLength, COUNT(focalLength), AgInternedExifCameraModel.value, AgInternedExifLens.value " +\
                  "FROM AgHarvestedExifMetadata, AgInternedExifCameraModel, AgInternedExifLens " +\
                  "WHERE cameraModelRef = AgInternedExifCameraModel.id_local AND "+\
                  "lensRef = AgInternedExifLens.id_local " +\
                  "GROUP BY focalLength, AgInternedExifCameraModel.value, AgInternedExifLens.value " +\
                  "ORDER BY focalLength ASC",db)

focalLength  COUNT(focalLength)                  value  \
0          7.300                   1   Canon PowerShot A630   
1          8.800                 119            DSC-RX100M3   
2          8.800                  70            DSC-RX100M3   
3          8.820                   3            DSC-RX100M3   
4          8.900                   1            DSC-RX100M3   
5          9.120                   1            DSC-RX100M3   
6          9.120                   1            DSC-RX100M3   
7          9.610                   2            DSC-RX100M3   
8          9.690                   1            DSC-RX100M3   
9          9.990                   1            DSC-RX100M3   
10        10.240                   1            DSC-RX100M3   
11        10.240                   1            DSC-RX100M3   
12        10.600                   1            DSC-RX100M3   
13        10.890                   1            DSC-RX100M3   
14        11.000                 333           Canon EOS 7D   
15        11.450                   4            DSC-RX100M3   
16        11.930                   1            DSC-RX100M3   
17        12.000                  24           Canon EOS 7D   
18        12.060                   1            DSC-RX100M3   
19        12.565                   1   Canon PowerShot A630   
20        13.000                  23           Canon EOS 7D   
21        13.340                   1            DSC-RX100M3   
22        14.000                  14           Canon EOS 7D   
23        14.300                   2            DSC-RX100M3   
24        14.700                   2            DSC-RX100M3   
25        14.700                   1            DSC-RX100M3   
26        15.000                  36           Canon EOS 7D   
27        16.000                 275           Canon EOS 7D   
28        16.000                  30           Canon EOS 7D   
29        17.000                 554  Canon EOS 5D Mark III   
..           ...                 ...                    ...   
208      180.000                  23  Canon EOS 5D Mark III   
209      180.000                  21           Canon EOS 7D   
210      182.000                   8           Canon EOS 7D   
211      185.000                  40  Canon EOS 5D Mark III   
212      185.000                  33           Canon EOS 7D   
213      190.000                  44  Canon EOS 5D Mark III   
214      190.000                  35           Canon EOS 7D   
215      190.000                   6           Canon EOS 7D   
216      195.000                  35  Canon EOS 5D Mark III   
217      195.000                  32           Canon EOS 7D   
218      195.000                  16           Canon EOS 7D   
219      200.000                 692  Canon EOS 5D Mark III   
220      200.000                1150           Canon EOS 7D   
221      203.000                   3           Canon EOS 7D   
222      210.000                   5           Canon EOS 7D   
223      217.000                  14           Canon EOS 7D   
224      225.000                   8           Canon EOS 7D   
225      230.000                   4           Canon EOS 7D   
226      238.000                   1  Canon EOS 5D Mark III   
227      238.000                   7           Canon EOS 7D   
228      245.000                  14           Canon EOS 7D   
229      260.000                  17           Canon EOS 7D   
230      265.000                   3           Canon EOS 7D   
231      273.000                   9           Canon EOS 7D   
232      280.000                   7  Canon EOS 5D Mark III   
233      280.000                 207           Canon EOS 7D   
234      300.000                 351  Canon EOS 5D Mark III   
235      300.000                1254           Canon EOS 7D   
236      300.000                  27  Canon EOS-1D Mark III   
237      300.000                1773   Canon EOS-1D Mark IV   

                           value  
0                    7.3-29.2 mm  
1               24-70mm F1.8